In [111]:
import numpy as np
class zmarray(np.ndarray):

    def __new__(cls, input_array, n=None, m=None, alpha=None):
        # Input array is an already formed ndarray instance
        # We first cast to be our class type
        obj = np.asarray(input_array).view(cls)
        obj.N = obj.shape[1]
        obj.alpha = alpha
        if n is None:
            obj.n = j2nm(range(obj.N), obj.alpha)[:, 0]
        else:
            obj.n = n
        if m is None:
            obj.m = j2nm(range(obj.N), obj.alpha)[:, 1]
        else:
            obj.m = m
        return obj

    def __getitem__(self, index):
        # Handling slicing
        result = super(zmarray, self).__getitem__(index)
        
        # If the result is not a scalar (i.e., still an array)
        if isinstance(result, np.ndarray):
            # Update n and m for column slicing
            if isinstance(index, tuple) and len(index) > 1:
                col_index = index[1]
                if isinstance(col_index, slice):
                    result.n = self.n[col_index]
                    result.m = self.m[col_index]
                    result.N = len(self.n[col_index])
                elif isinstance(col_index, int):
                    result.n = self.n[col_index:col_index+1]
                    result.m = self.m[col_index:col_index+1]
                    result.N = 1
            # For row slicing or single dimension slicing, n and m remain the same
            else:
                result.n = self.n
                result.m = self.m

        return result
            
            
    def __array_finalize__(self, obj):
        if obj is None: return
        self.n = getattr(obj, 'n', None)
        self.m = getattr(obj, 'm', None)
        self.alpha = getattr(obj, 'alpha', None)
        self.N = getattr(obj, 'N', None)

In [112]:
a = np.random.random((20, 10))
m = np.arange(10)
n = -np.arange(10)
aa = zmarray(a, n=n, m=m)

In [115]:
bb = aa[:, 5]

In [75]:
class myarray(np.ndarray):

    def __new__(cls, input_array):
        # Input array is an already formed ndarray instance
        # We first cast to be our class type
        obj = np.asarray(input_array).view(cls)
        obj.col_inds = np.arange(input_array.shape[1])
        return obj
        

    def __array_finalize__(self, obj):
        if obj is None: return
        self.col_inds = getattr(obj, 'col_inds', None)

In [79]:
a = np.random.random((100, 10))
aa = myarray(a)

bb = aa[:, 0:3]
print(bb.col_inds)

[0 1 2 3 4 5 6 7 8 9]


In [77]:
bb = aa[:, 0:3]
print(bb.col_inds)

[0 1 2 3 4 5 6 7 8 9]


In [78]:
bb.col_inds

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])